In [1]:
# MIA brain tumor project model

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True     
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Data loading and preprocessing

data_transforms = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Testing': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Assigning the data loc

data_dir = 'Input/tumor_ct'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Testing', 'Training']}



dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['Training', 'Testing']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['Training', 'Testing']}

class_names = image_datasets['Training'].classes



In [5]:
#Training the model function

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):

    since = time.time()

    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            
            for phase in ['Training', 'Testing']:
                if phase == 'Training':
                    model.train()  
                else:
                    model.eval()   

                running_loss = 0.0
                running_corrects = 0

                
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    
                    optimizer.zero_grad()

                    
                    with torch.set_grad_enabled(phase == 'Training'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        
                        if phase == 'Training':
                            loss.backward()
                            optimizer.step()

                   
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'Training':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'Testing' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [7]:
# Transfer Learning using ResNet50 and feature extraction

model_ft = models.resnet50(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [9]:
# Fitting the model and defining number of epochs

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
Training Loss: 0.9027 Acc: 0.6571
Testing Loss: 1.4784 Acc: 0.6447

Epoch 1/49
----------
Training Loss: 0.7047 Acc: 0.7418
Testing Loss: 1.5286 Acc: 0.7513

Epoch 2/49
----------
Training Loss: 0.5292 Acc: 0.8132
Testing Loss: 1.1493 Acc: 0.7234

Epoch 3/49
----------
Training Loss: 0.4873 Acc: 0.8286
Testing Loss: 1.2460 Acc: 0.7640

Epoch 4/49
----------
Training Loss: 0.4350 Acc: 0.8491
Testing Loss: 1.8392 Acc: 0.7208

Epoch 5/49
----------
Training Loss: 0.3974 Acc: 0.8641
Testing Loss: 1.0390 Acc: 0.7995

Epoch 6/49
----------
Training Loss: 0.3435 Acc: 0.8864
Testing Loss: 1.3561 Acc: 0.7386

Epoch 7/49
----------
Training Loss: 0.2470 Acc: 0.9185
Testing Loss: 1.3486 Acc: 0.7513

Epoch 8/49
----------
Training Loss: 0.2115 Acc: 0.9303
Testing Loss: 1.1176 Acc: 0.7640

Epoch 9/49
----------
Training Loss: 0.1745 Acc: 0.9408
Testing Loss: 1.4487 Acc: 0.7563

Epoch 10/49
----------
Training Loss: 0.1965 Acc: 0.9321
Testing Loss: 1.2721 Acc: 0.7665

Epoch 11/

In [ ]:
# saving the model for future to add in app

torch.save(model_ft.state_dict(), 'tumormod.pth')